In [1]:
# import json

# with open('bot_tmp.log', encoding='utf-8') as f:
#     lines = [line.split(" --- ") for line in f.readlines()]

#     res = {'files': []}

#     for idx, line in enumerate(lines):
#         entry = {
#             'id': idx + 1,
#             'type': line[1].strip()[1:-1],
#             'name': line[0].strip()[1:-1],
#             'uploaded_date': line[2].strip()[1:-1]
#         }

#         res['files'].append(entry)

#     with open('files_info.json', 'w', encoding='utf-8') as f:
#         json.dump(res, f, indent=4, ensure_ascii=False)